In [ ]:
# --- IMPORTAÇÕES (BIBLIOTECAS) --- #
import requests
import time
from datetime import datetime

In [ ]:
# --- CONFIGURAÇÕES GLOBAIS --- #
MOEDA_DESTINO = "BRL"

ALERTA_ALTA = 6.00
ALERTA_BAIXA = 4.00

ARQUIVO_SAIDA = "cotacoes.txt"
INTERVALO_VERIFICACAO = 180  # 3 minutos em segundos

In [2]:
def escolha_moeda():
    """
    Solicita ao usuário a escolha da moeda para monitoramento.
    
    Returns:
        str: Código da moeda em maiúsculo (ex: 'USD', 'BTC')
    """
    while True:  # Loop até obter uma moeda válida
        try:
            moeda = input("Digite o código da moeda (USD, BTC, EUR, etc.): ")
            moeda = moeda.upper()  # 🔧 CORREÇÃO: Atribuindo o resultado do upper()
            
            # Validação básica - verifica se não está vazio e tem formato adequado
            if len(moeda) >= 3 and moeda.isalpha():
                return moeda
            else:
                print("❌ Código inválido! Digite um código de moeda válido (ex: USD, BTC)")
                
        except KeyboardInterrupt:
            print("\n🛑 Operação cancelada pelo usuário.")
            return None

def validar_moeda_api(moeda_origem):
    """
    Valida se a moeda existe na API antes de iniciar o monitoramento.
    
    Args:
        moeda_origem (str): Código da moeda a ser validada
        
    Returns:
        bool: True se a moeda for válida, False caso contrário
    """
    try:
        url = f"https://economia.awesomeapi.com.br/last/{moeda_origem}-{MOEDA_DESTINO}"
        resposta = requests.get(url, verify=False, timeout=10)
        
        if resposta.status_code == 200:
            dados = resposta.json()
            # Verifica se a chave da moeda existe no retorno
            if moeda_origem + MOEDA_DESTINO in dados:
                return True
        
        print(f"❌ Moeda '{moeda_origem}' não encontrada na API!")
        return False
        
    except requests.RequestException as e:
        print(f"❌ Erro ao validar moeda: {e}")
        return False

def registro_arquivo_cotacoes(moeda_origem, cotacao_atual):
    """
    Registra a cotação atual em arquivo de texto para histórico.
    
    Args:
        moeda_origem (str): Código da moeda de origem
        cotacao_atual (float): Valor atual da cotação
    """
    try:
        data_hora = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
        linha = f"{data_hora} | {moeda_origem}->{MOEDA_DESTINO} | R$ {cotacao_atual:.2f}\n"

        with open(ARQUIVO_SAIDA, 'a', encoding='utf-8') as arquivo:
            arquivo.write(linha)
            
        print(f"📝 Cotação registrada no arquivo: {ARQUIVO_SAIDA}")
        
    except IOError as e:
        print(f"❌ Erro ao salvar no arquivo: {e}")

def exibir_mensagem_alerta(cotacao_atual, limite_cambio, tipo_alerta):
    """
    Exibe mensagem de alerta quando limites são ultrapassados.
    
    Args:
        cotacao_atual (float): Cotação atual da moeda
        limite_cambio (float): Limite que foi ultrapassado
        tipo_alerta (str): 'ALTA' ou 'BAIXA'
    """
    # 🔧 CORREÇÃO: Agora a função realmente UTILIZA as mensagens
    print("=" * 50)
    print(f"🚨 ALERTA DE {tipo_alerta}: Moeda a R$ {cotacao_atual:.2f}")
    print("=" * 50)
    
    mensagem_detalhada = f"""
    📊 RELATÓRIO DE MONITORAMENTO
    
    ⚠️  ATENÇÃO: Limite de {tipo_alerta.lower()} ultrapassado!
    
    💰 Valor Atual: R$ {cotacao_atual:.2f}
    🎯 Limite Definido: R$ {limite_cambio:.2f}
    📅 Data/Hora: {datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}
    
    🤖 Monitoramento finalizado automaticamente.
    """
    
    print(mensagem_detalhada)

def obter_cotacao(moeda_origem):
    """
    Obtém a cotação atual da moeda via API.
    
    Args:
        moeda_origem (str): Código da moeda de origem
        
    Returns:
        float or None: Valor da cotação ou None em caso de erro
    """
    try:
        url = f"https://economia.awesomeapi.com.br/last/{moeda_origem}-{MOEDA_DESTINO}"
        resposta = requests.get(url, verify=False, timeout=10)
        
        if resposta.status_code == 200:
            dados = resposta.json()
            cotacao_atual = float(dados[moeda_origem + MOEDA_DESTINO]['bid'])
            return cotacao_atual
        else:
            print(f"❌ Erro na API. Status: {resposta.status_code}")
            return None
            
    except requests.RequestException as e:
        print(f"❌ Erro de conexão: {e}")
        return None
    except (KeyError, ValueError) as e:
        print(f"❌ Erro ao processar dados da API: {e}")
        return None

In [ ]:
def iniciar_monitoramento():
    """
    Função principal que coordena todo o processo de monitoramento.
    """
    print("🚀 Iniciando Sistema de Monitoramento de Criptomoedas")
    print("-" * 55)
    
    # 1. Escolha e validação da moeda
    moeda_origem = escolha_moeda()
    if not moeda_origem:  # Se usuário cancelou
        return
    
    # 2. Validação na API
    if not validar_moeda_api(moeda_origem):
        print("🛑 Monitoramento cancelado - moeda inválida.")
        return
    
    print(f"✅ Moeda {moeda_origem} validada com sucesso!")
    print(f"📊 Monitorando {moeda_origem}/{MOEDA_DESTINO}")
    print(f"🔺 Alerta de ALTA: R$ {ALERTA_ALTA:.2f}")
    print(f"🔻 Alerta de BAIXA: R$ {ALERTA_BAIXA:.2f}")
    print(f"⏱️  Verificação a cada {INTERVALO_VERIFICACAO//60} minutos")
    print("-" * 55)
    
    # 3. Loop de monitoramento
    contador_verificacoes = 0
    
    try:
        while True:
            contador_verificacoes += 1
            print(f"\n🔄 Verificação #{contador_verificacoes} - {datetime.now().strftime('%H:%M:%S')}")
            
            # Obtém cotação atual
            cotacao_atual = obter_cotacao(moeda_origem)
            
            if cotacao_atual is None:
                print("⚠️ Falha ao obter cotação. Tentando novamente em breve...")
                time.sleep(30)  # Aguarda menos tempo em caso de erro
                continue
            
            # Registra no arquivo
            registro_arquivo_cotacoes(moeda_origem, cotacao_atual)
            print(f"💹 Cotação atual: R$ {cotacao_atual:.2f}")
            
            # 🔧 CORREÇÃO: Lógica otimizada com elif
            if cotacao_atual >= ALERTA_ALTA:
                exibir_mensagem_alerta(cotacao_atual, ALERTA_ALTA, "ALTA")
                break
                
            elif cotacao_atual <= ALERTA_BAIXA:
                exibir_mensagem_alerta(cotacao_atual, ALERTA_BAIXA, "BAIXA")
                break
                
            else:
                print("✅ Cotação dentro do esperado. Continuando monitoramento...")
            
            # Aguarda próxima verificação
            print(f"⏳ Próxima verificação em {INTERVALO_VERIFICACAO//60} minutos...")
            time.sleep(INTERVALO_VERIFICACAO)
            
    except KeyboardInterrupt:
        print("\n\n🛑 Monitoramento interrompido pelo usuário.")
        print("📊 Dados coletados salvos em:", ARQUIVO_SAIDA)
    
    except Exception as e:
        print(f"\n❌ Erro inesperado: {e}")
        print("📊 Dados coletados salvos em:", ARQUIVO_SAIDA)

# 🚀 EXECUÇÃO DO PROGRAMA
if __name__ == "__main__":
    iniciar_monitoramento()

🚀 Iniciando Sistema de Monitoramento de Criptomoedas
-------------------------------------------------------


C:\Users\ana.leandro\AppData\Roaming\Python\Python313\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'economia.awesomeapi.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\ana.leandro\AppData\Roaming\Python\Python313\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'economia.awesomeapi.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ Moeda USD validada com sucesso!
📊 Monitorando USD/BRL
🔺 Alerta de ALTA: R$ 6.00
🔻 Alerta de BAIXA: R$ 4.00
⏱️  Verificação a cada 3 minutos
-------------------------------------------------------

🔄 Verificação #1 - 12:59:19
📝 Cotação registrada no arquivo: cotacoes.txt
💹 Cotação atual: R$ 5.37
✅ Cotação dentro do esperado. Continuando monitoramento...
⏳ Próxima verificação em 3 minutos...

🔄 Verificação #2 - 13:02:19


C:\Users\ana.leandro\AppData\Roaming\Python\Python313\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'economia.awesomeapi.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


📝 Cotação registrada no arquivo: cotacoes.txt
💹 Cotação atual: R$ 5.37
✅ Cotação dentro do esperado. Continuando monitoramento...
⏳ Próxima verificação em 3 minutos...
